In [2]:
import numpy as np
import pandas as pd
import os
import gc
import warnings
warnings.filterwarnings("ignore") # ignore python warnings of deprecation

df = pd.read_csv('Risk Averse.csv', na_values=['#NAME?']) # '#NAME?' in the datafile will be converted to NaN

In [3]:
df.shape

(887379, 35)

In [4]:
df.head(1)

,id,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,total_acc,total_pymnt,A,B,...,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding,int_rate
0,1077501,5000,24000.0,27.65,0.0,1.0,9.0,5861.071414,0,1,...,0,0,0,0,0,0,0,0,0,10.65


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 35 columns):
id                    887379 non-null int64
loan_amnt             887379 non-null int64
annual_inc            887379 non-null float64
dti                   887379 non-null float64
delinq_2yrs           887379 non-null float64
inq_last_6mths        887379 non-null float64
total_acc             887379 non-null float64
total_pymnt           887379 non-null float64
A                     887379 non-null int64
B                     887379 non-null int64
C                     887379 non-null int64
D                     887379 non-null int64
E                     887379 non-null int64
F                     887379 non-null int64
G                     887379 non-null int64
MORTGAGE              887379 non-null int64
NONE                  887379 non-null int64
OTHER                 887379 non-null int64
OWN                   887379 non-null int64
RENT                  887379 non-null int

In [6]:
#Removing features which has got null values above 75%
nulls = pd.DataFrame(round(df.isnull().sum()/len(df.index)*100,2),columns=['null_percent'])
drop_cols = nulls[nulls['null_percent']>75.0].index
df.drop(drop_cols, axis=1, inplace=True)

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X = df.iloc[:, :-1].values
y = df.iloc[:, -1:].values
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

ValueError: Unknown label type: (array([[10.65],
       [15.27],
       [15.96],
       ...,
       [15.99],
       [19.99],
       [11.99]]),)

In [10]:
#Splitting the dataset into Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [11]:
from sklearn.linear_model import LinearRegression
#Fitting Simple Linear Regression to the training set
regressor = LinearRegression()
regressor.fit(X_train, y_train)
#Predicting the Test set result
y_pred = regressor.predict(X_test)

In [12]:
print(y_pred[0:5])

[[11.00611951]
 [ 8.2261983 ]
 [11.59562264]
 [10.98661531]
 [14.58043658]]


In [13]:
print(y_test[0:5])

[[ 9.17]
 [ 5.99]
 [10.16]
 [12.99]
 [14.64]]


In [14]:
#Mean Absolute Percentage Error for Linear Regression Model
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
#Calling the function
mean_absolute_percentage_error(y_test, y_pred)

7.844451389092959